In [30]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [31]:
path = r'C:\Users\i.tur\Desktop\drive\ds\jdsz3-zczasem\ProjektML'
filename = '/Dane/Carseats.csv'
os.chdir(path)

In [32]:
df = pd.read_csv(path + filename)
df.head()

,Unnamed: 0,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,1,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,2,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,3,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,4,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,5,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [33]:
df.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [34]:
df[['Urban', 'US']] = df[['Urban', 'US']].replace({'Yes':1, 'No':0})

In [35]:
categorical_cols = ['ShelveLoc']
df_categorical_dummies = pd.get_dummies(df[categorical_cols])
print(df_categorical_dummies.head())

   ShelveLoc_Bad  ShelveLoc_Good  ShelveLoc_Medium
0              1               0                 0
1              0               1                 0
2              0               0                 1
3              0               0                 1
4              1               0                 0


In [36]:
df = pd.concat([df, df_categorical_dummies], axis = 1)
df.drop(categorical_cols, axis = 1, inplace = True)
df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,Urban,US,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium
0,9.50,138,73,11,276,120,42,17,1,1,1,0,0
1,11.22,111,48,16,260,83,65,10,1,1,0,1,0
2,10.06,113,35,10,269,80,59,12,1,1,0,0,1
3,7.40,117,100,4,466,97,55,14,1,1,0,0,1
4,4.15,141,64,3,340,128,38,13,1,0,1,0,0


In [37]:
df.to_csv('Dane/Carseats_Clean.csv', index=False)

In [38]:
df_cls = df
df_cls['Sales_class'] = ''

przedzialy = {}

df_cls['Sales_class'] = np.where(df_cls['Sales'] < df_cls['Sales'].quantile(0.333), 'LOW', np.where(df_cls['Sales'] < df_cls['Sales'].quantile(0.666), 'MED', 'HIGH'))

df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,Urban,US,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium,Sales_class
0,9.50,138,73,11,276,120,42,17,1,1,1,0,0,HIGH
1,11.22,111,48,16,260,83,65,10,1,1,0,1,0,HIGH
2,10.06,113,35,10,269,80,59,12,1,1,0,0,1,HIGH
3,7.40,117,100,4,466,97,55,14,1,1,0,0,1,MED
4,4.15,141,64,3,340,128,38,13,1,0,1,0,0,LOW


## KNN regresja dla sales jako wartość docelowa

In [39]:
%matplotlib inline
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_boston

In [40]:
df.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,Urban,US,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000,0.705000,0.645000,0.240000,0.212500,0.547500
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528,0.456614,0.479113,0.427618,0.409589,0.498362
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000,1.000000,1.000000,0.000000,0.000000,1.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [51]:
#podział danych na testowe i treningowe
X = df.drop(['Sales','Sales_class'], axis=1).values.astype('float')
y = df['Sales'].values.astype('float') 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=444)


array([ 6.1 ,  7.22,  7.49,  4.11,  4.62,  6.63,  6.41,  4.9 ,  4.2 ,
        9.33,  7.63,  3.67,  8.97,  8.07, 13.28,  9.01,  8.71,  2.07,
        8.23, 10.31,  5.3 , 11.62,  7.64,  8.31,  7.78,  4.38,  8.77,
       11.27,  6.5 ,  2.52,  9.32,  5.57,  6.93, 10.59,  5.97,  4.97,
        5.93,  4.95,  7.91,  7.56,  5.47,  8.21,  7.43, 11.82, 13.36,
       12.66,  8.47,  8.25,  9.03,  9.7 ,  7.81,  7.32,  5.35,  7.38,
        4.36,  6.52, 10.43,  5.17,  4.19,  4.53, 11.99,  8.85, 11.27,
        5.9 , 12.04,  3.02,  6.88,  9.35,  7.36,  7.54,  6.52,  0.91,
        6.5 ,  6.8 , 12.57, 11.28, 10.49,  6.98, 11.85,  8.39, 10.26,
       10.51,  6.64,  6.85, 10.08,  4.95, 12.49,  1.82,  5.36,  5.86,
        8.77,  9.24, 11.22,  5.87,  5.32, 12.13, 10.98,  9.  , 10.66,
        2.23,  2.05,  8.78,  8.8 ,  6.67,  7.45, 13.39,  4.69,  9.58,
        7.71,  5.08, 10.14,  7.8 ,  5.71,  4.83,  3.58,  6.97,  9.39,
       16.27,  5.68,  4.12,  5.27,  4.17, 13.44,  6.68,  8.55,  6.14,
        9.62,  4.1 ,

In [54]:
# normalizacja
scaler = MinMaxScaler()
scaler.fit(X_train)
norm_x_train = scaler.transform(X_train)
norm_x_test = scaler.transform(X_test)

[[0.89411765 0.42424242 0.         0.07459677 0.67114094 0.56363636
  0.75       1.         0.         1.         0.         0.        ]
 [0.43529412 0.92929293 0.08333333 0.2358871  0.85234899 0.27272727
  0.625      0.         1.         0.         1.         0.        ]
 [0.69411765 0.98989899 0.25       0.33467742 0.81208054 0.18181818
  0.375      1.         1.         0.         0.         1.        ]
 [0.42352941 0.48484848 0.45833333 0.16532258 0.55033557 0.92727273
  0.25       0.         1.         0.         0.         1.        ]
 [0.51764706 0.75757576 0.         0.92741935 0.76510067 0.47272727
  0.25       1.         0.         0.         0.         1.        ]]


In [55]:
#pierwszy model
b_reg = KNeighborsRegressor(n_neighbors=4)
b_reg.fit(norm_x_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                    weights='uniform')

In [57]:
# test bez dopasowania parametrów
preds = b_reg.predict(norm_x_test)
r2_score(y_test, preds)

0.3093757891346607

In [58]:
# próba poprawy poprzez zmianę parametrów
best_score = 0.0
best_params = {}

for neighbors in range(1, 101):
    for w in ["uniform", "distance"]:
        for p in [1, 2]:
            regressor = KNeighborsRegressor(n_neighbors=neighbors, p=p, weights=w)
            scores = cross_val_score(regressor, norm_x_train, y_train, cv=5)
            score = scores.mean()
            if score > best_score:
                best_score = score
                best_params["neighbors"] = neighbors
                best_params["weights"] = w
                best_params["p"] = p
print(best_score)
print(best_params)

0.4302205596194476
{'neighbors': 20, 'weights': 'distance', 'p': 1}


In [72]:
# wynik po dopasowaniu parametrów
regressor = KNeighborsRegressor(n_neighbors=best_params["neighbors"],
                                p=best_params["p"],
                                weights=best_params["weights"])
regressor.fit(norm_x_train, y_train)
preds = regressor.predict(norm_x_test)
r2_score(y_test, preds)

0.37661221309973003

## KNN klasyfikacja dla sales_class jako wartość docelowa

In [104]:
#podział danych na testowe i treningowe
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
X = df_cls.drop(['Sales','Sales_class'], axis=1).values.astype('float')
y = df_cls['Sales_class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=444)

In [105]:
scaler = MinMaxScaler()
scaler.fit(X_train)
norm_x_train = scaler.transform(X_train)
norm_x_test = scaler.transform(X_test)

In [106]:
b_classifier = KNeighborsClassifier(n_neighbors=4)
b_classifier.fit(norm_x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [107]:
preds = b_classifier.predict(norm_x_test)
accuracy_score(y_test, preds)

0.50625

In [109]:
best_score = 0.0
best_params = {}

for neighbors in range(1, 50):
    for w in ["uniform", "distance"]:
        for p in [1, 2]:
            classifier = KNeighborsClassifier(n_neighbors=neighbors, p=p, weights=w)
            scores = cross_val_score(classifier, norm_x_train, y_train, cv=5)
            score = scores.mean()
            if score > best_score:
                best_score = score
                best_params["neighbors"] = neighbors
                best_params["weights"] = w
                best_params["p"] = p
print(best_score)
print(best_params)

0.5856209150326797
{'neighbors': 31, 'weights': 'uniform', 'p': 1}


In [110]:
classifier = KNeighborsClassifier(n_neighbors=31,
                                  p=1,
                                  weights='uniform')
classifier.fit(norm_x_train, y_train)
preds = classifier.predict(norm_x_test)
accuracy_score(y_test, preds)

0.528125